In [1]:
import warnings
warnings.filterwarnings("ignore")
%load_ext autoreload
%autoreload 2
from qiskit_metal import Dict, MetalGUI, designs

from qiskit_metal.qlibrary.qubits.transmon_pocket import TransmonPocket
from qiskit_metal.qlibrary.qubits.transmon_pocket_6 import TransmonPocket6
from qiskit_metal.qlibrary.tlines.meandered import RouteMeander
from qiskit_metal.qlibrary.tlines.straight_path import RouteStraight
from qiskit_metal.qlibrary.terminations.launchpad_wb_coupled import LaunchpadWirebondCoupled
from qiskit_metal.qlibrary.terminations.short_to_ground import ShortToGround
from qiskit_metal.qlibrary.couplers.coupled_line_tee import CoupledLineTee
from qiskit_metal.qlibrary.user_components.left_coupler import LeftCoupler
from qiskit_metal.qlibrary.user_components.right_coupler import RightCoupler
from qiskit_metal.qlibrary.user_components.straightdown_coupler import StraightDownCoupler
from qiskit_metal.qlibrary.user_components.straightup_coupler import StraightUpCoupler

In [36]:
design = designs.DesignPlanar({}, overwrite_enabled=True)
gui = MetalGUI(design)
design.chips.main.size['size_x']='6mm'
design.chips.main.size['size_y']='6mm'

In [3]:
design.chips

{'main': {'material': 'silicon',
  'layer_start': '0',
  'layer_end': '2048',
  'size': {'center_x': '0.0mm',
   'center_y': '0.0mm',
   'center_z': '0.0mm',
   'size_x': '6mm',
   'size_y': '6mm',
   'size_z': '-750um',
   'sample_holder_top': '890um',
   'sample_holder_bottom': '1650um'}}}

In [37]:

cp0 = dict(down = dict(loc_W=1, loc_H=-1, pad_width='120um'))


# q0=[TransmonPocket6(design, 'Q0', options=dict(pos_x='-2000um',
#                                             pos_y='-2000um',
#                                             chip='main',
#                                             connection_pads=cp0))]
q0=[TransmonPocket6(design, 'Q0', options=dict(pos_x='-1300um',
                                            pos_y='-470um',
                                            chip='main'))]
                                            
qubits=q0
q0x=float(qubits[0].options.pos_x.strip('um'))
q0y=float(qubits[0].options.pos_y.strip('um'))


c0 = StraightUpCoupler(design, "c0", options=dict(pos_x=f"{q0x-150}um",
                                              pos_y=f"{q0y+140}um",
                                              pad_width="150um",
                                              pad_height="50um",
                                              cpw_stub_height="200um",
                                              chip="main"))
design.rebuild()
gui.rebuild()
gui.autoscale()

In [38]:
lpL=LaunchpadWirebondCoupled(design, 'lpL',
                                         options = dict(chip="main",
                                         pos_x = '-1700um',
                                         pos_y = '1300um',
                                         lead_length = '30um'))
lpR=LaunchpadWirebondCoupled(design, 'lpR',
                                         options = dict(chip="main",
                                         pos_x = '1700um',
                                         pos_y = '1300um',
                                         orientation='180',
                                         lead_length = '30um'))                                        

In [39]:
stg=ShortToGround(design, 'stg', options=Dict(chip="main",
                                                pos_x='-0.585mm',
                                                pos_y='0mm',
                                                orientation='270'))

In [40]:
fl0=RouteStraight(design, 'fl0',options=Dict(chip="main", pin_inputs=Dict(start_pin=Dict(
                                                            component='lpL',
                                                            pin='tie'),
                                                            end_pin=Dict(
                                                            component='lpR',
                                                            pin='tie'))))


# Readout resonator

In [26]:
from collections import OrderedDict
jogs=OrderedDict()
def connect(component_name: str, component1: str, pin1: str, component2: str, pin2: str,
            length: str, start_jogged_extension: OrderedDict, end_jogged_extension: OrderedDict,
            spacing: str, asymmetry='0 um',  start_straight='0 um', end_straight='0 um', flip=False):
    """Connect two pins with a CPW."""
    myoptions = Dict(
        chip="main",
        pin_inputs=Dict(
            start_pin=Dict(
                component=component1,
                pin=pin1),
            end_pin=Dict(
                component=component2,
                pin=pin2)),
        lead=Dict(
            start_straight=start_straight,
            end_straight=end_straight,
            start_jogged_extension=start_jogged_extension,
            end_jogged_extension=end_jogged_extension),
        total_length=length,
        fillet = '99.9um',
        meander=dict(asymmetry=asymmetry,
                     spacing=spacing))
    myoptions.meander.lead_direction_inverted = 'true' if flip else 'false'
    return RouteMeander(design, component_name, myoptions)

# Resonator length

In [27]:
from qiskit_metal.analyses.em.cpw_calculations import guided_wavelength

def find_resonator_length(frequency, line_width, line_gap, N): 
    #frequency in GHz
    #line_width/line_gap in um
    #N -> 2 for lambda/2, 4 for lambda/4
    
    [lambdaG, etfSqrt, q] = guided_wavelength(frequency*10**9, line_width*10**-6,
                                              line_gap*10**-6, 750*10**-6, 200*10**-9)
    return str(lambdaG/N*10**3)+" mm"

In [41]:
resonator_length_4=[]
for i in range(3):
    a=find_resonator_length(7.2+i*0.2,10,6,4)
    a=a.strip('mm')
    a=float(a)-0.35
    resonator_length_4.append(a)
print(resonator_length_4)


[3.876627341935142, 3.7623941699207104, 3.654173270104565]


In [42]:
jogs1=OrderedDict()
jogs1[0]=['R','350um']
jogs1[1]=['R', '400um']
jogs1[2]=['L','515um']


In [45]:
rr0=connect('rr0', 'c0', 'in','stg', 'short',
            resonator_length_4[1],jogs1,jogs, 
            '400um','0um', '1388um', '100um')
            

# EPR Analysis

In [46]:
from qiskit_metal.analyses.quantization import EPRanalysis
eig_rd = EPRanalysis(design, "hfss")
hfss = eig_rd.sim.renderer

In [47]:
hfss.start()

INFO 04:05PM [connect_project]: Connecting to Ansys Desktop API...
INFO 04:05PM [load_ansys_project]: 	Opened Ansys App
INFO 04:05PM [load_ansys_project]: 	Opened Ansys Desktop v2018.0.0
INFO 04:05PM [load_ansys_project]: 	Opened Ansys Project
	Folder:    C:/Users/kimje/Documents/Ansoft/
	Project:   Project2
INFO 04:05PM [connect_design]: No active design found (or error getting active design).
INFO 04:05PM [connect]: 	 Connected to project "Project2". No design detected


True

In [48]:
hfss.activate_ansys_design("Planar_V2", 'eigenmode')
hfss.render_design(['Q0','c0','lpL','lpR','stg','rr0','fl0'],[],
                    box_plus_buffer=False)

04:05PM 05s WARNING [activate_ansys_design]: The design_name=Planar_V2 was not in active project.  Designs in active project are: 
[].  A new design will be added to the project.  
INFO 04:05PM [connect_design]: 	Opened active design
	Design:    Planar_V2 [Solution type: Eigenmode]
WARNING 04:05PM [connect_setup]: 	No design setup detected.
WARNING 04:05PM [connect_setup]: 	Creating eigenmode default setup.
INFO 04:05PM [get_setup]: 	Opened setup `Setup`  (<class 'pyEPR.ansys.HfssEMSetup'>)


In [49]:
hfss.close()


True

# LOM Analysis

In [50]:
from qiskit_metal.analyses.quantization import LOManalysis
c1 = LOManalysis(design, "q3d")

In [51]:
c1.sim.setup.max_passes = 10
c1.sim.setup_update(freq_ghz='5.83',solution_order = 'Medium', auto_increase_solution_order = 'False')
c1.sim.setup

{'name': 'Setup',
 'reuse_selected_design': True,
 'reuse_setup': True,
 'freq_ghz': '5.83',
 'save_fields': False,
 'enabled': True,
 'max_passes': 10,
 'min_passes': 2,
 'min_converged_passes': 2,
 'percent_error': 0.5,
 'percent_refinement': 30,
 'auto_increase_solution_order': 'False',
 'solution_order': 'Medium',
 'solver_type': 'Iterative'}

In [52]:
c1.sim.run(components=['Q0','c0'], open_terminations=[('c0', 'in')])
c1.sim._get_results_from_renderer()
c1.sim.capacitance_matrix

INFO 04:47PM [connect_project]: Connecting to Ansys Desktop API...
INFO 04:47PM [load_ansys_project]: 	Opened Ansys App
INFO 04:47PM [load_ansys_project]: 	Opened Ansys Desktop v2018.0.0
INFO 04:47PM [load_ansys_project]: 	Opened Ansys Project
	Folder:    C:/Users/kimje/Documents/Ansoft/
	Project:   planarV2
INFO 04:47PM [connect_design]: 	Opened active design
	Design:    Planar_V2 [Solution type: Eigenmode]
INFO 04:47PM [get_setup]: 	Opened setup `Setup`  (<class 'pyEPR.ansys.HfssEMSetup'>)
INFO 04:47PM [connect]: 	Connected to project "planarV2" and design "Planar_V2" 😀 

INFO 04:47PM [connect_design]: 	Opened active design
	Design:    Design_q3d [Solution type: Q3D]
WARNING 04:47PM [connect_setup]: 	No design setup detected.
WARNING 04:47PM [connect_setup]: 	Creating Q3D default setup.
INFO 04:47PM [get_setup]: 	Opened setup `Setup`  (<class 'pyEPR.ansys.AnsysQ3DSetup'>)
INFO 04:47PM [get_setup]: 	Opened setup `Setup`  (<class 'pyEPR.ansys.AnsysQ3DSetup'>)
INFO 04:47PM [analyze]: An

,ground_main_plane,pad_bot_Q0,pad_top_Q0,quad_coupler_c0
ground_main_plane,173.980,-47.0270,-41.044,-28.2990
pad_bot_Q0,-47.027,87.5790,-34.769,-2.1893
pad_top_Q0,-41.044,-34.7690,97.840,-18.9260
quad_coupler_c0,-28.299,-2.1893,-18.926,50.6100


In [53]:
c1.setup.junctions = Dict({'Lj': 11, 'Cj': 2})
c1.setup.freq_readout = 7.09
c1.setup.freq_bus = []
c1.run_lom()
c1.lumped_oscillator_all

[1, 2] [3]
Predicted Values

Transmon Properties
f_Q 5.686268 [GHz]
EC 303.604815 [MHz]
EJ 14.854145 [GHz]
alpha -349.374392 [MHz]
dispersion 14.395518 [KHz]
Lq 10.995564 [nH]
Cq 63.800793 [fF]
T1 41.695062 [us]

**Coupling Properties**

tCqbus1 -7.328533 [fF]
gbus1_in_MHz -115.711202 [MHz]
χ_bus1 -3.860641 [MHz]
1/T1bus1 3817.117313 [Hz]
T1bus1 41.695062 [us]
Bus-Bus Couplings


,fQ,EC,EJ,alpha,dispersion,gbus,chi_in_MHz,χr MHz,gr MHz
1,6.080523,350.304387,14.854145,-408.623621,53.667055,[-93.02906726368401],[-4.982767851232566],4.982768,93.029067
2,6.061672,347.983968,14.854145,-405.645758,50.594621,[-109.10844292851404],[-6.6073028520605686],6.607303,109.108443
3,5.979658,337.992017,14.854145,-392.86356,38.976439,[-107.98543621843154],[-5.544727150128193],5.544727,107.985436
4,5.903114,328.816965,14.854145,-381.184373,30.350375,[-112.70245644554512],[-5.262136930901238],5.262137,112.702456
5,5.809223,317.759522,14.854145,-367.182334,22.127096,[-114.53457326450693],[-4.623797590306809],4.623798,114.534573
6,5.76884,313.07006,14.854145,-361.268146,19.253038,[-113.85926309833472],[-4.272385586866052],4.272386,113.859263
7,5.723583,307.861687,14.854145,-354.716269,16.434209,[-115.32060419663661],[-4.070785798119941],4.070786,115.320604
8,5.70427,305.654236,14.854145,-351.944711,15.348743,[-115.59463585704192],[-3.965086647756785],3.965087,115.594636
9,5.686268,303.604815,14.854145,-349.374392,14.395518,[-115.71120233932909],[-3.860641139203433],3.860641,115.711202


In [30]:
c1.sim.close()